In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Sat Jun 11 04:47:36 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   52C    P0    30W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Download dependencies

In [ ]:
%%capture
!pip install datasets==1.18.3
!pip install transformers==4.17.0
!pip install jiwer

In [ ]:
!apt install git-lfs

Reading package lists... Done
Building dependency tree       
Reading state information... Done
git-lfs is already the newest version (2.3.4-1).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import re
import nltk
from nltk import word_tokenize, pos_tag

In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

# Preparing Data

In [ ]:
import os
fpaths = []
labels = []
spoken = []
for f in os.listdir('/content/drive/MyDrive/Child_Dataset-20220609T161342Z-001/Child_Dataset'):
    for w in os.listdir('/content/drive/MyDrive/Child_Dataset-20220609T161342Z-001/Child_Dataset/' + f):
        fpaths.append('/content/drive/MyDrive/Child_Dataset-20220609T161342Z-001/Child_Dataset/' + f + '/' + w)
        labels.append(f)
        if f not in spoken:
            spoken.append(f)
print('Words spoken:', spoken)

Words spoken: ['Zebras eat hay and giraffes eat carrots', 'when they see the frog out the jar', 'with a the pot on his head', 'Workers use parts of the bottles to make clothing', 'Without gravity on Earth you would not be able to walk', 'which i bought in nerul', 'When you jump rope you get exercise that helps your heart', 'Wild animals find their own food', 'who wanted to eat me', 'When water covers streets and goes into houses it is called a flood', 'when they get dressed they look for the frog', 'when the boy and the dog are going to the bed the frog gets out of the jar', 'When the shell breaks open a butterfly comes out', 'Volcanoes made some high mountains there too', 'when he_s in water and he_s like byyyyeeee', 'when the boy and the dog wake up the frog isn_t in the jar', 'uuuhhh the dog hm jumps out of the window', 'when the kid and the dog is asleep', 'when the rabbit came the lion roared why are you late', 'When someone wants a drink they add water to the pouch of their choic

# Preprocess the Data

In [ ]:
def preprocess_data(tweet):
    user_name = '@[A-Za-z0-9]+' # Pattern for matching the user names in the tweet
    hash_tag = '#[A-Za-z0-9]+' # Pattern for matching the hasgtags in the tweer
    url = 'https?:\/\/[A-Za-z0-9\.\/\-]+' # Pattern for matching the URLs in the tweet
    
    x = tweet.lower()  # Lower casing all the characters
    #x = re.sub(user_name, '@USER', x)  # Replace the username with @USER word -> Optional
    x = re.sub(user_name, '', x) # Replace the username with an empty characrer
    #x = re.sub(has_tag,'HASH',x) # Replace the hashtags with HASH word -> Optional
    x = re.sub(hash_tag,'',x) # Replace the hashtags with an empty characrer
    #x = re.sub(url, 'URL', x) # Replace the URLs with URL word -> Optional
    x = re.sub(url, '', x) # Replace the URLs with an empty characrer
    x = re.sub("[^a-zA-Z0-9]", " ",x) # Replace all the characters except alphabets and digits from the tweet
    x = re.sub("rt\s+",'',x) # Replace the retweet character "rt" from the tweet
    x = re.sub(r'\b\d+\b', '', x)
    x = x.replace('copy','')
  
    tokens = word_tokenize(x) # Tokenization; splitting the text into tokens 

    return(" ".join(tokens)) # Combine the tokenized words to form a sentence

In [ ]:
clean_labels = []
for i in labels:
  x = preprocess_data(i)
  clean_labels.append(x)

In [ ]:
clean_labels

['zebras eat hay and giraffes eat carrots',
 'zebras eat hay and giraffes eat carrots',
 'zebras eat hay and giraffes eat carrots',
 'when they see the frog out the jar',
 'when they see the frog out the jar',
 'when they see the frog out the jar',
 'when they see the frog out the jar',
 'when they see the frog out the jar',
 'with a the pot on his head',
 'with a the pot on his head',
 'with a the pot on his head',
 'with a the pot on his head',
 'with a the pot on his head',
 'workers use parts of the bottles to make clothing',
 'workers use parts of the bottles to make clothing',
 'workers use parts of the bottles to make clothing',
 'workers use parts of the bottles to make clothing',
 'workers use parts of the bottles to make clothing',
 'workers use parts of the bottles to make clothing',
 'workers use parts of the bottles to make clothing',
 'workers use parts of the bottles to make clothing',
 'workers use parts of the bottles to make clothing',
 'workers use parts of the bottl

In [ ]:
import os
import json
import librosa
import soundfile as sf
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    fpaths, 
    clean_labels, 
    test_size=0.2
)

In [ ]:
audio = []
for wav in X_train:
  x,sr=librosa.load(wav,16000)
  audio.append(x)

/usr/local/lib/python3.7/dist-packages/librosa/core/audio.py:165: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")
/usr/local/lib/python3.7/dist-packages/librosa/core/audio.py:165: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")
/usr/local/lib/python3.7/dist-packages/librosa/core/audio.py:165: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")
/usr/local/lib/python3.7/dist-packages/librosa/core/audio.py:165: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")
/usr/local/lib/python3.7/dist-packages/librosa/core/audio.py:165: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")
/usr/local/lib/python3.7/dist-packages/librosa/cor

In [ ]:
sample_rate = []
for i in range(1,2753):
  x = 16000
  sample_rate.append(x)

In [ ]:
import datasets
from datasets import Dataset
import pandas as pd

In [ ]:
import pandas as pd
train_df = pd.DataFrame.from_dict({"path": X_train, "text": y_train,'audio':audio,'sample_rate': sample_rate})

In [ ]:
audio1 = []
for wav in X_test:
  x,sr=librosa.load(wav,16000)

  audio1.append(x)

/usr/local/lib/python3.7/dist-packages/librosa/core/audio.py:165: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")
/usr/local/lib/python3.7/dist-packages/librosa/core/audio.py:165: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")
/usr/local/lib/python3.7/dist-packages/librosa/core/audio.py:165: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")
/usr/local/lib/python3.7/dist-packages/librosa/core/audio.py:165: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")
/usr/local/lib/python3.7/dist-packages/librosa/core/audio.py:165: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")
/usr/local/lib/python3.7/dist-packages/librosa/cor

In [ ]:
sample_rate1 = []
for i in range(1,690):
  x = 16000
  sample_rate1.append(x)

In [ ]:
test_df = pd.DataFrame.from_dict({"path": X_test, "text": y_test,'audio':audio1,'sample_rate': sample_rate1})

In [ ]:
import datasets
from datasets import Dataset
train_dataset = Dataset.from_dict(train_df)
test_dataset = Dataset.from_dict(test_df)
dataset = datasets.DatasetDict({"train":train_dataset,"test":test_dataset})

In [ ]:
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    display(HTML(df.to_html()))

In [ ]:
# displaying the 10 random elements of the dataset
show_random_elements(dataset['train'].remove_columns(["path",'audio','sample_rate']), num_examples=20)

,text
0,nine
1,then then the deer is like happy
2,and they was in the water and they was wet
3,two
4,nine
5,then the rat looked at the dog
6,and he has fallen the glass
7,the scientists will try to save them
8,and the dog is looking in in the bee s nest
9,out of water penguins must waddle and hop across the ice


In [ ]:
import re
chars_to_ignore_regex = '[\,\?\.\!\-\;\:\"\“\%\‘\”\�]'

def remove_special_characters(batch):
    batch["text"] = re.sub(chars_to_ignore_regex, '', batch["text"]).lower() + " "
    return batch

In [ ]:
dataset = dataset.map(remove_special_characters)

Parameter 'function'=<function remove_special_characters at 0x7f3fbe4ae4d0> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


0ex [00:00, ?ex/s]

0ex [00:00, ?ex/s]

In [ ]:
show_random_elements(dataset["train"].remove_columns(["path","audio","sample_rate"]))

,text
0,the dog is in front of the horse
1,other animals will not eat the stink bug because it tastes bad
2,when they see the frog out the jar
3,and he s mad
4,two
5,the dog is on top of the shed
6,he licks him cos he wanted to put the frog in
7,hop back in the pound
8,then the boy fell over and all of the bees was flying
9,they play and jump in the water


In [ ]:
def extract_all_chars(batch):
  all_text = " ".join(batch["text"])
  vocab = list(set(all_text))
  return {"vocab": [vocab], "all_text": [all_text]}

In [ ]:
vocabs = dataset.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=dataset.column_names["train"])

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:
vocab_list = list(set(vocabs["train"]["vocab"][0]) | set(vocabs["test"]["vocab"][0]))

In [ ]:
vocab_dict = {v: k for k, v in enumerate(vocab_list)}
vocab_dict

{' ': 0,
 '0': 19,
 '1': 16,
 '5': 24,
 'a': 13,
 'b': 12,
 'c': 6,
 'd': 20,
 'e': 18,
 'f': 26,
 'g': 17,
 'h': 28,
 'i': 1,
 'j': 14,
 'k': 2,
 'l': 23,
 'm': 5,
 'n': 3,
 'o': 9,
 'p': 22,
 'q': 15,
 'r': 11,
 's': 8,
 't': 29,
 'u': 4,
 'v': 7,
 'w': 10,
 'x': 27,
 'y': 21,
 'z': 25}

In [ ]:
vocab_dict["|"] = vocab_dict[" "]
del vocab_dict[" "]

In [ ]:
vocab_dict["[UNK]"] = len(vocab_dict)
vocab_dict["[PAD]"] = len(vocab_dict)
len(vocab_dict)

32

In [ ]:
import json
with open('vocab.json', 'w') as vocab_file:
    json.dump(vocab_dict, vocab_file)

In [ ]:
from transformers import Wav2Vec2CTCTokenizer

tokenizer = Wav2Vec2CTCTokenizer("./vocab.json", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")

In [ ]:
from transformers import Wav2Vec2FeatureExtractor

feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=True)

In [ ]:
from transformers import Wav2Vec2Processor

processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)

In [ ]:
processor.save_pretrained("wav2vec2-large-xlsr-childD-demo")

In [ ]:
import IPython.display as ipd
import numpy as np
import random

rand_int = random.randint(0, len(dataset["train"]))

print(dataset["train"][rand_int]["text"])
ipd.Audio(data=np.asarray(dataset["train"][rand_int]["audio"]), autoplay=True, rate=16000)

the dog and the kid is looking at the the the frog 


In [ ]:
rand_int = random.randint(0, len(dataset["train"]))

print("Target text:", dataset["train"][rand_int]["text"])
print("Input array shape:", np.asarray(dataset["train"][rand_int]["audio"]).shape)
print("Sampling rate:", dataset["train"][rand_int]["sample_rate"])

Target text: because he can t find the frog 
Input array shape: (43407,)
Sampling rate: 16000


In [ ]:
def prepare_dataset(batch):
    # check that all files have the correct sampling rate
    assert (
        len(set(batch["sample_rate"])) == 1
    ), f"Make sure all inputs have the same sampling rate of {processor.feature_extractor.sampling_rate}."

    batch["input_values"] = processor(batch["audio"], sampling_rate=batch["sample_rate"][0]).input_values
    
    with processor.as_target_processor():
        batch["labels"] = processor(batch["text"]).input_ids
    return batch

In [ ]:
dataset = dataset.map(prepare_dataset, remove_columns=dataset.column_names["train"], batch_size=4, num_proc=4, batched=True)

/usr/local/lib/python3.7/dist-packages/transformers/feature_extraction_utils.py:169: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  tensor = as_tensor(value)
/usr/local/lib/python3.7/dist-packages/transformers/feature_extraction_utils.py:169: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  tensor = as_tensor(value)
/usr/local/lib/python3.7/dist-packages/transformers/feature_extraction_utils.py:169: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths o

# Define tokenizer 

In [ ]:
import torch

from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
        max_length (:obj:`int`, `optional`):
            Maximum length of the ``input_values`` of the returned list and optionally padding length (see above).
        max_length_labels (:obj:`int`, `optional`):
            Maximum length of the ``labels`` returned list and optionally padding length (see above).
        pad_to_multiple_of (:obj:`int`, `optional`):
            If set will pad the sequence to a multiple of the provided value.
            This is especially useful to enable the use of Tensor Cores on NVIDIA hardware with compute capability >=
            7.5 (Volta).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True
    max_length: Optional[int] = None
    max_length_labels: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    pad_to_multiple_of_labels: Optional[int] = None

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lenghts and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                max_length=self.max_length_labels,
                pad_to_multiple_of=self.pad_to_multiple_of_labels,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch

In [ ]:
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

In [ ]:
from datasets import load_dataset, load_metric
wer_metric = load_metric("wer")

Downloading:   0%|          | 0.00/1.90k [00:00<?, ?B/s]

In [ ]:
def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    wer = wer_metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

In [ ]:
from transformers import Wav2Vec2ForCTC

model = Wav2Vec2ForCTC.from_pretrained(
    "facebook/wav2vec2-large-xlsr-53", 
    attention_dropout=0.1,
    hidden_dropout=0.1,
    feat_proj_dropout=0.0,
    mask_time_prob=0.05,
    layerdrop=0.1,
    gradient_checkpointing=True, 
    ctc_loss_reduction="mean", 
    pad_token_id=processor.tokenizer.pad_token_id,
    vocab_size=len(processor.tokenizer)
)

Downloading:   0%|          | 0.00/1.73k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.18G [00:00<?, ?B/s]

Some weights of the model checkpoint at facebook/wav2vec2-large-xlsr-53 were not used when initializing Wav2Vec2ForCTC: ['project_q.bias', 'project_hid.bias', 'quantizer.weight_proj.bias', 'quantizer.weight_proj.weight', 'project_q.weight', 'project_hid.weight', 'quantizer.codevectors']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-large-xlsr-53 and are newly initialized: ['lm_head.bias', 'lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to u

In [ ]:
model.freeze_feature_extractor()

/usr/local/lib/python3.7/dist-packages/transformers/models/wav2vec2/modeling_wav2vec2.py:1712: FutureWarning: The method `freeze_feature_extractor` is deprecated and will be removed in Transformers v5.Please use the equivalent `freeze_feature_encoder` method instead.
  FutureWarning,


In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
  output_dir="/content/drive/MyDrive/wav2vec2-large-xlsr-childD-demo",
  # output_dir="./wav2vec2-large-xlsr-turkish-demo",
  group_by_length=True,
  per_device_train_batch_size=4,
  gradient_accumulation_steps=2,
  evaluation_strategy="steps",
  num_train_epochs=30,
  fp16=True,
  save_steps=400,
  eval_steps=400,
  logging_steps=400,
  learning_rate=3e-4,
  warmup_steps=500,
  save_total_limit=2,
)

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=dataset['train'],
    eval_dataset=dataset['test'],
    tokenizer=processor.feature_extractor,
)

Using amp half precision backend


In [ ]:
trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 2752
  Num Epochs = 30
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 2
  Total optimization steps = 10320


Step,Training Loss,Validation Loss,Wer
400,6.649200,3.111422,1.000000
800,3.010900,2.933664,1.000000
1200,2.887600,2.823949,0.998329
1600,2.782700,2.678381,0.948402
2000,2.635700,2.589919,0.943597
2400,2.524500,2.320722,0.942762
2800,2.161100,1.847485,0.875287
3200,1.546300,1.106171,0.688740
3600,1.030500,0.689297,0.537915
4000,0.736000,0.494404,0.435764


***** Running Evaluation *****
  Num examples = 689
  Batch size = 8
Saving model checkpoint to /content/drive/MyDrive/wav2vec2-large-xlsr-childD-demo/checkpoint-400
Configuration saved in /content/drive/MyDrive/wav2vec2-large-xlsr-childD-demo/checkpoint-400/config.json
Model weights saved in /content/drive/MyDrive/wav2vec2-large-xlsr-childD-demo/checkpoint-400/pytorch_model.bin
Feature extractor saved in /content/drive/MyDrive/wav2vec2-large-xlsr-childD-demo/checkpoint-400/preprocessor_config.json
***** Running Evaluation *****
  Num examples = 689
  Batch size = 8
Saving model checkpoint to /content/drive/MyDrive/wav2vec2-large-xlsr-childD-demo/checkpoint-800
Configuration saved in /content/drive/MyDrive/wav2vec2-large-xlsr-childD-demo/checkpoint-800/config.json
Model weights saved in /content/drive/MyDrive/wav2vec2-large-xlsr-childD-demo/checkpoint-800/pytorch_model.bin
Feature extractor saved in /content/drive/MyDrive/wav2vec2-large-xlsr-childD-demo/checkpoint-800/preprocessor_conf

TrainOutput(global_step=10320, training_loss=1.1260336378748104, metrics={'train_runtime': 17769.3577, 'train_samples_per_second': 4.646, 'train_steps_per_second': 0.581, 'total_flos': 1.214852652670396e+19, 'train_loss': 1.1260336378748104, 'epoch': 30.0})

In [ ]:
processor = Wav2Vec2Processor.from_pretrained("/content/drive/MyDrive/wav2vec2-large-xlsr-childD-demo")

loading feature extractor configuration file /content/drive/MyDrive/wav2vec2-large-xlsr-childD-demo/preprocessor_config.json
Feature extractor Wav2Vec2FeatureExtractor {
  "do_normalize": true,
  "feature_extractor_type": "Wav2Vec2FeatureExtractor",
  "feature_size": 1,
  "padding_side": "right",
  "padding_value": 0.0,
  "processor_class": "Wav2Vec2Processor",
  "return_attention_mask": true,
  "sampling_rate": 16000
}

Didn't find file /content/drive/MyDrive/wav2vec2-large-xlsr-childD-demo/added_tokens.json. We won't load it.
loading file /content/drive/MyDrive/wav2vec2-large-xlsr-childD-demo/vocab.json
loading file /content/drive/MyDrive/wav2vec2-large-xlsr-childD-demo/tokenizer_config.json
loading file None
loading file /content/drive/MyDrive/wav2vec2-large-xlsr-childD-demo/special_tokens_map.json
Adding <s> to the vocabulary
Adding </s> to the vocabulary
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
def map_to_result(batch):
  with torch.no_grad():
    input_values = torch.tensor(batch["input_values"], device="cuda").unsqueeze(0)
    logits = model(input_values).logits

  pred_ids = torch.argmax(logits, dim=-1)
  batch["pred_str"] = processor.batch_decode(pred_ids)[0]
  batch["text"] = processor.decode(batch["labels"], group_tokens=False)
  
  return batch

In [ ]:
results = dataset["test"].map(map_to_result, remove_columns=dataset["test"].column_names)

0ex [00:00, ?ex/s]

In [ ]:
print("Test WER: {:.3f}".format(wer_metric.compute(predictions=results["pred_str"], references=results["text"])))

Test WER: 0.134


In [ ]:
show_random_elements(results)

,pred_str,text
0,one,one
1,animals li a loong ie toast and nearbycoceans,animals live along the coast and in nearby oceans
2,seven,seven
3,and they went back home,and they went back home
4,a lmore people recycle cans bottles and paper,now a lot more people recycle cans bottles and paper
5,and and he skipped out the the the bottle,and and he skipped out the the the bottle
6,i think it s owl yeah,i think it s owl yeah
7,and then he s like no,and then he s like no
8,the dog is looking in a beehive,the dog is looking in a beehive
9,the rabbit replied i saw another lied on my way,the rabbit replied i saw another lied on my way
